In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#House price prediction in boston

Pytorch의 MLP를 이용한 집값 예측 모델 구현 (regression)
  
주요 모듈 목록  
**torch.utils.data.Dataset:** 데이터셋을 getitem method에서 한개씩 텐서형태로 반환하도록 구성하는 class  
**torch.utils.data.DataLoader:** batch processing을 위해 dataset에서 반환되는 데이터를 n개의 batch_size만큼 묶어서 반환하도록 하는 class  
**torch.nn.Module:** 딥러닝 모델을 구현한 class, forward method를 통해 입력된 텐서 데이터를 딥러닝 연산하여 결과 반환  
**torch.nn.MSELoss:** loss 연산을 위한 class이며 backward method를 통해 gradient계산  
**torch.optim.Adam:** 계산된 gradient를 update해주어 feadient descent를 진행하는 class  

# Dataset class구현  
  
dataset(Housingdata.csv) : https://www.kaggle.com/datasets/altavish/boston-housing-dataset

target: MEDV(집값)  
input feature: MEDV를 제외한 모든 값  
전체 데이터 약 500개중 400개까지 train data로 사용, 나머지 100개는 test data  

각 data를 torch.utils.data.Dataset을 통해 하나씩 load할 수 있는 class구현  
이후 torch.utils.data.DataLoader를 통해 batch개씩 변환

torch.utils.data.Dataset의 주요 method

__ init __: 클래스를 오브젝트로 생성할때 불러와지는 함수, 클래스에서 필요한 인스턴스(데이터셋, 데이터경로)등을 생성  
__ len __: 해당 클래스에서 다루는 dataset의 길이를 반환하는 함수  
__ getitem __(index): index에 해당하는 데이터 하나를 tensor형태로 반환하는 함수

In [9]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch

from sklearn.preprocessing import MinMaxScaler


data_path = "/content/drive/MyDrive/nlp-open-tutorial/2일차_배포/dataset/HousingData.csv"
batch_size = 4

#torch.utils.data.Dataset을 상속하여 Dataset class선언
class myDataset(Dataset):
  #오브젝트를 선언할때 불러오는 함수, superclass(부모클래스)의 init을 실행해 주어야함
  def __init__(self, df_data) -> None:
    super().__init__()
    self.y = df_data.loc[:,["MEDV"]]
    self.x = df_data.drop(["MEDV"], axis=1)

  #list 형태의 class를 만들때 필수로 사용되는 함수, 전체 길이를 알아야 인덱싱이 가능
  def __len__(self):
    return len(self.y)

  #index에 해당하는 데이터를 반환해주는 함수
  def __getitem__(self, index):
    data = torch.Tensor(self.x.loc[index,:])
    target = torch.FloatTensor(self.y.loc[index,:])
    return data, target

#data load후 train(400개)/test(나머지)데이터를 분할
data_df = pd.read_csv(data_path).dropna()
train_data_df = data_df.loc[:400,:].reset_index()
test_data_df = data_df.loc[400:,:].reset_index()

#train dataset, test dataset을 각각 선언
trainDataset = myDataset(train_data_df)
testDataset = myDataset(test_data_df)

#torch.utils.data.DataLoader을 사용 dataloader 선언, 이를 통해 batch processing
trainDataloader = DataLoader(trainDataset, batch_size)
testDataloader = DataLoader(testDataset, batch_size)

#잘 작동하는지 test
for i in trainDataset:
  print("dataset test")
  print(i)
  break

#잘 작동하는지 test
for i in trainDataloader:
  print("data loader test")
  data = i[0]
  target = i[1]

  print("data")
  print(data)
  print(data.shape)
  print("\ntarget")
  print(target)
  print(target.shape)
  break


dataset test
(tensor([0.0000e+00, 6.3200e-03, 1.8000e+01, 2.3100e+00, 0.0000e+00, 5.3800e-01,
        6.5750e+00, 6.5200e+01, 4.0900e+00, 1.0000e+00, 2.9600e+02, 1.5300e+01,
        3.9690e+02, 4.9800e+00]), tensor([24.]))
data loader test
data
tensor([[0.0000e+00, 6.3200e-03, 1.8000e+01, 2.3100e+00, 0.0000e+00, 5.3800e-01,
         6.5750e+00, 6.5200e+01, 4.0900e+00, 1.0000e+00, 2.9600e+02, 1.5300e+01,
         3.9690e+02, 4.9800e+00],
        [1.0000e+00, 2.7310e-02, 0.0000e+00, 7.0700e+00, 0.0000e+00, 4.6900e-01,
         6.4210e+00, 7.8900e+01, 4.9671e+00, 2.0000e+00, 2.4200e+02, 1.7800e+01,
         3.9690e+02, 9.1400e+00],
        [2.0000e+00, 2.7290e-02, 0.0000e+00, 7.0700e+00, 0.0000e+00, 4.6900e-01,
         7.1850e+00, 6.1100e+01, 4.9671e+00, 2.0000e+00, 2.4200e+02, 1.7800e+01,
         3.9283e+02, 4.0300e+00],
        [3.0000e+00, 3.2370e-02, 0.0000e+00, 2.1800e+00, 0.0000e+00, 4.5800e-01,
         6.9980e+00, 4.5800e+01, 6.0622e+00, 3.0000e+00, 2.2200e+02, 1.8700e+01,
     

# Deep Learning Model 구현  
torch.nn.Module을 이용하여 모델 구현

1st hidden layer의 feature는 100개  
2nd hidden layer의 feature는 10개인 모델을 구현한다.  

torch.nn.Linear: perceptron의 weighted sum과 같이 linaer regression연산을 하는 calss  
torch.nn.ReLU: ReLU activation function을 수행하는 class  

torch.nn.Module의 주요 method  
__ init __: 클래스를 오브젝트로 생성할때 불러와지는 함수, 클래스에서 필요한 인스턴스(사용할 deep learning layer, activation function, 등)등을 생성  
__ forward __(data): 입력받은 data를 딥러닝 모델을 통해 결과를 예측하여 반환하는 class  

**각 딥러닝 레이어 연산 중 차원수를 확인하고 잘 맞춰줄 것**  
참고 document(해당 사이트의 shape를 확인하고 tensor형태 결정)  
nn.Linear: https://pytorch.org/docs/stable/generated/torch.nn.Linear.html  
nn.ReLU: https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html

In [14]:
from torch import nn

#딥러닝 모델을 작성하기위한 모듈
class myModel(nn.Module):
  #오브젝트를 선언할 때 불러와지는 함수 일반적으로 모델에서 사용될 각 레이어들이 포함됨
  def __init__(self) -> None:
      super().__init__()
      #input_feature:14,  1st_hidden: 100, 2nd_hidden: 10, output: 1의 형태에 맞는 linear layer들을 선언, activation으로 Relu사용
      self.linear = nn.Linear(14, 100)
      self.linear2 = nn.Linear(100, 10)
      self.linear3 = nn.Linear(10,1)
      self.relu = nn.ReLU()

  #데이터를 입력받고 딥러닝 연산후 결과를 반환하는 함수
  def forward(self, x):
      x = self.linear(x)
      x = self.relu(x)
      x = self.linear2(x)
      x = self.relu(x)
      x = self.linear3(x)

      return x

#작성한 모델 선언
model = myModel()

#잘 작동하는지 test
for i in trainDataloader:
  print("model test")
  data = i[0]
  target = i[1]

  print("input data")
  print(data)
  print(data.shape)
  print("\noutput predict")
  print(model(data))
  print(model(data).shape)
  print("\nground thruth")
  print(target)
  print(target.shape)
  break


model test
input data
tensor([[0.0000e+00, 6.3200e-03, 1.8000e+01, 2.3100e+00, 0.0000e+00, 5.3800e-01,
         6.5750e+00, 6.5200e+01, 4.0900e+00, 1.0000e+00, 2.9600e+02, 1.5300e+01,
         3.9690e+02, 4.9800e+00],
        [1.0000e+00, 2.7310e-02, 0.0000e+00, 7.0700e+00, 0.0000e+00, 4.6900e-01,
         6.4210e+00, 7.8900e+01, 4.9671e+00, 2.0000e+00, 2.4200e+02, 1.7800e+01,
         3.9690e+02, 9.1400e+00],
        [2.0000e+00, 2.7290e-02, 0.0000e+00, 7.0700e+00, 0.0000e+00, 4.6900e-01,
         7.1850e+00, 6.1100e+01, 4.9671e+00, 2.0000e+00, 2.4200e+02, 1.7800e+01,
         3.9283e+02, 4.0300e+00],
        [3.0000e+00, 3.2370e-02, 0.0000e+00, 2.1800e+00, 0.0000e+00, 4.5800e-01,
         6.9980e+00, 4.5800e+01, 6.0622e+00, 3.0000e+00, 2.2200e+02, 1.8700e+01,
         3.9463e+02, 2.9400e+00]])
torch.Size([4, 14])

output predict
tensor([[-3.4231],
        [-2.4632],
        [-2.8585],
        [-3.2171]], grad_fn=<AddmmBackward0>)
torch.Size([4, 1])

ground thruth
tensor([[24.0000],
 

# 작성한 dataset과 model을 이용하여 deep learning process 구현

pytorch 딥러닝 프로세스
1. dataset, model선언
2. dataset과 model을 통한 결과 예측
3. 예측된 결과를 통해 **loss연산** 및 **loss.backward**
4. **optimizer.step()**를 사용하여 graident update

주요 오브젝트  
torch.nn.MSELoss: 예측값과 정답을 통해 MSE값을 반환하는 class  
torch.optim.Adam: Adam optimizer를 통해 gradient update를 수행하는 class


각 오브젝트의 입력과 선언은 다음 doc 참조:  
MSELoss: https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html  
Adam: https://pytorch.org/docs/stable/generated/torch.optim.Adam.html

In [15]:
from torch.optim import Adam
from torch.nn import MSELoss

#학습을 위한 optimizer와 loss function 설정
lf = MSELoss()
optimizer = Adam(model.parameters(), lr=0.001)

#100번의 epoch을 실행
for e in range(100):
  print("\nepoch ", e)
  epoch_loss = 0

  #선언한 model을 학습 가능한 상태로 변경
  model.train()

  #모든 train data에 대해서 학습
  for i in trainDataloader:
    #매 배치에 대한 gradient 계산 이전에 optimizer에 저장된 이전 batch의 gradient를 삭제(초기화)
    optimizer.zero_grad()

    data = i[0]
    target = i[1]

    #결과 도출
    output = model(data)

    #loss 계산 (MSE는 output과 target의 shape이 일치하도록)
    loss = lf(output, target)

    #backpropagation
    loss.backward()

    #gradient update
    optimizer.step()

    epoch_loss += loss.item()

  print("train loss", epoch_loss/len(trainDataloader))

  #model이 학습되지 않는 상태로 변경
  model.eval()
  test_loss = 0

  #gradient를 계산하지 않도록 하여 cost낭비 방지
  with torch.no_grad():
    #모든 test dataset에 대해서 결과연산
    for i in testDataloader:
      data = i[0]
      target = i[1]

      output = model(data)

      loss = lf(output, target)
      test_loss += loss.item()

  print("test loss", test_loss/len(testDataloader))




epoch  0
train loss 126.8013574443286
test loss 65.75581132797967

epoch  1
train loss 84.81891114500505
test loss 92.38695117405483

epoch  2
train loss 71.48581759160078
test loss 125.60539345514206

epoch  3
train loss 80.74466441401952
test loss 121.91030665806362

epoch  4
train loss 88.68818681904033
test loss 145.44529015677315

epoch  5
train loss 90.7303034685835
test loss 139.93490155537924

epoch  6
train loss 89.55470282518411
test loss 146.6666997273763

epoch  7
train loss 84.59585257459291
test loss 140.08889843168714

epoch  8
train loss 80.95090159811551
test loss 147.8440024058024

epoch  9
train loss 72.70341113926489
test loss 100.95087342035202

epoch  10
train loss 66.12874949657464
test loss 150.00431242443267

epoch  11
train loss 56.744451739365545
test loss 149.63891092936197

epoch  12
train loss 51.848193124879764
test loss 142.11482002621605

epoch  13
train loss 49.32077111850811
test loss 154.69649505615234

epoch  14
train loss 46.363729919813856
test l